In [1]:
%matplotlib inline
import itertools
import math
import time

import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image

import matplotlib.pyplot as plt
from IPython import display

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
latent_size = 64
hidden_size = 256
image_size = 784
num_epochs = 200
batch_size = 100
sample_dir = 'samples'

In [4]:
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [5]:
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5],   # 1 for greyscale channels
                                     std=[0.5])])

In [6]:
mnist = torchvision.datasets.MNIST(root='data',
                                   train=True,
                                   transform=transform,
                                   download=True)

In [7]:
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                          batch_size=batch_size, 
                                          shuffle=True)

In [16]:
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid())

In [17]:
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh())

In [18]:
D = D.to(device)
G = G.to(device)

In [14]:
criterion = nn.BCELoss()

d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

# d_optimizer = torch.optim.SGD(D.parameters(), lr=0.02)
# g_optimizer = torch.optim.SGD(G.parameters(), lr=0.02)

# d_optimizer = torch.optim.SGD(D.parameters(), lr=0.02, momentum=.9)
# g_optimizer = torch.optim.SGD(G.parameters(), lr=0.02, momentum=.9)

# d_optimizer = torch.optim.SGD(D.parameters(), lr=0.02, momentum=.9, nesterov=True)
# g_optimizer = torch.optim.SGD(G.parameters(), lr=0.02, momentum=.9, nesterov=True)

# d_optimizer = torch.optim.Adagrad(D.parameters(), lr=0.02)
# g_optimizer = torch.optim.Adagrad(G.parameters(), lr=0.02)

# d_optimizer = torch.optim.Adadelta(D.parameters(), lr=0.02)
# g_optimizer = torch.optim.Adadelta(G.parameters(), lr=0.02)

# d_optimizer = torch.optim.RMSprop(D.parameters(), lr=0.02)
# g_optimizer = torch.optim.RMSprop(G.parameters(), lr=0.02)

In [12]:
def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

In [ ]:
# Start training
num_test_samples = 16
test_noise = torch.autograd.Variable(torch.randn(num_test_samples, latent_size).cuda())
G_losses = []
D_losses = []

# size_figure_grid = int(math.sqrt(num_test_samples))
# fig, ax = plt.subplots(size_figure_grid, size_figure_grid, figsize=(6, 6))
# for i, j in itertools.product(range(size_figure_grid), range(size_figure_grid)):
#     ax[i,j].get_xaxis().set_visible(False)
#     ax[i,j].get_yaxis().set_visible(False)

# num_fig = 0

total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        images = images.reshape(batch_size, -1).to(device)
        
        # Create the labels which are later used as input for the BCE loss
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # ================================================================== #
        #                      Train the discriminator                       #
        # ================================================================== #

        # Compute BCE_Loss using real images where BCE_Loss(x, y): - y * log(D(x)) - (1-y) * log(1 - D(x))
        # Second term of the loss is always zero since real_labels == 1
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        # Compute BCELoss using fake images
        # First term of the loss is always zero since fake_labels == 0
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # Backprop and optimize
        d_loss = d_loss_real + d_loss_fake
        reset_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # ================================================================== #
        #                        Train the generator                         #
        # ================================================================== #

        # Compute loss with fake images
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        
        # We train G to maximize log(D(G(z)) instead of minimizing log(1-D(G(z)))
        # For the reason, see the last paragraph of section 3. https://arxiv.org/pdf/1406.2661.pdf
        g_loss = criterion(outputs, real_labels)
        
        # Backprop and optimize
        reset_grad()
        g_loss.backward()
        g_optimizer.step()

        if epoch % 2 == 0:
          G_losses.append(g_loss.item())
          D_losses.append(d_loss.item())
        
        # if (i+1) % 200 == 0:
        #     print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
        #           .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), real_score.mean().item(), fake_score.mean().item()))
        if (i+1) % 200 == 0:
            # test_images = G(test_noise)
            
            # for k in range(num_test_samples):
            #     i = k // 4
            #     j = k % 4
            #     ax[i,j].cla()
            #     ax[i,j].imshow(test_images[k,:].data.cpu().numpy().reshape(28, 28), cmap='Greys')
            # display.clear_output(wait=True)
            # display.display(plt.gcf())
            
            # plt.savefig('results/mnist-gan-%03d.png'%num_fig)
            # num_fig += 1
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}'.format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), real_score.mean().item(), fake_score.mean().item()))
    
    # # Save real images
    # if (epoch+1) == 1:
    #     images = images.reshape(images.size(0), 1, 28, 28)
    #     save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))
    
    # # Save sampled images
    # fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    # save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))

Epoch [0/200], Step [200/600], d_loss: 1.3829, g_loss: 0.6733, D(x): 0.51, D(G(z)): 0.51
Epoch [0/200], Step [400/600], d_loss: 1.3833, g_loss: 0.6729, D(x): 0.51, D(G(z)): 0.51
Epoch [0/200], Step [600/600], d_loss: 1.3869, g_loss: 0.6722, D(x): 0.51, D(G(z)): 0.51
Epoch [1/200], Step [200/600], d_loss: 1.3841, g_loss: 0.6726, D(x): 0.51, D(G(z)): 0.51
Epoch [1/200], Step [400/600], d_loss: 1.3844, g_loss: 0.6720, D(x): 0.51, D(G(z)): 0.51
Epoch [1/200], Step [600/600], d_loss: 1.3845, g_loss: 0.6724, D(x): 0.51, D(G(z)): 0.51
Epoch [2/200], Step [200/600], d_loss: 1.3877, g_loss: 0.6723, D(x): 0.51, D(G(z)): 0.51
Epoch [2/200], Step [400/600], d_loss: 1.3898, g_loss: 0.6730, D(x): 0.51, D(G(z)): 0.51
Epoch [2/200], Step [600/600], d_loss: 1.3844, g_loss: 0.6723, D(x): 0.51, D(G(z)): 0.51
Epoch [3/200], Step [200/600], d_loss: 1.3861, g_loss: 0.6725, D(x): 0.51, D(G(z)): 0.51
Epoch [3/200], Step [400/600], d_loss: 1.3815, g_loss: 0.6728, D(x): 0.51, D(G(z)): 0.51
Epoch [3/200], Step [

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()